# Transformers Interpret NER Example

### Setup

In [1]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

c:\Users\pablo\DFKI\transformers-interpret\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import NER model

In [2]:
MODEL_PATH = 'dslim/bert-base-NER'
model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)


We can explore the possible entity tags that this model classifies each token into:

In [3]:
model.config.id2label

{0: 'O',
 1: 'B-MISC',
 2: 'I-MISC',
 3: 'B-PER',
 4: 'I-PER',
 5: 'B-ORG',
 6: 'I-ORG',
 7: 'B-LOC',
 8: 'I-LOC'}


Import the **TokenClassificationExplainer** from the *transformers interpret* package. 

In [4]:
from transformers_interpret import TokenClassificationExplainer

Create an instance of the explainer providing the model and the tokenizer

In [5]:
ner_explainer = TokenClassificationExplainer(model=model, tokenizer=tokenizer)

### Basic usage

Call the explainer to generate the attributions. In the NER task, for each token of the sentence, the predicted class is considered as the target and the attributions of the rest of the tokens are calculated. This can take up to one or two minutes. In the following sections we will see how to limit the computation of the attributions to only certain tokens or only certain NER labels.

In [6]:
sample_text = "We visited Paris last weekend, where Emmanuel Macron lives."

In [18]:
attributions = ner_explainer(sample_text)

The explainer returns a dictionary with the predicted label and the attribtuion scores for every token.

In [19]:
attributions

{'[CLS]': {'label': 'O',
  'attribution_scores': [('[CLS]', 0.0),
   ('we', -0.2197283334110176),
   ('visited', -0.006874791837538396),
   ('paris', 0.18982619777915935),
   ('last', -0.21827055690979064),
   ('weekend', 0.3766208981865305),
   (',', 0.4892531296167725),
   ('where', 0.05182857620198211),
   ('emmanuel', 0.03630020660142918),
   ('macro', 0.1923832558634223),
   ('##n', -0.06223924004251359),
   ('lives', 0.27938115094347166),
   ('.', -0.6031927350485571),
   ('[SEP]', 0.0)]},
 'we': {'label': 'O',
  'attribution_scores': [('[CLS]', 0.0),
   ('we', 0.8284940046000231),
   ('visited', 0.10991536556232688),
   ('paris', -0.1125062256706944),
   ('last', -0.15386756933829704),
   ('weekend', -0.005880473809398968),
   (',', 0.004343130287280704),
   ('where', -0.28342447030227175),
   ('emmanuel', -0.41207122570713284),
   ('macro', -0.039487737627438534),
   ('##n', 0.052480672360112925),
   ('lives', 0.10229347071494145),
   ('.', -0.014845616401730032),
   ('[SEP]', 

**Note the alogirthm used to calcualte attributions are Layer Integreated Gradients to read more about them click [here](https://captum.ai/docs/algorithms)**

####  Visualizing explanations

With a single call to the `visualize()` method we get a nice inline display of what inputs are causing the activations to fire that led to classifying each of the tokens into a particular class. 

In [9]:
html = ner_explainer.visualize()

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
O,O (1.00),[CLS],1.13,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),We,0.16,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),visited,1.15,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
B-LOC,B-LOC (1.00),Paris,1.03,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),last,1.46,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),weekend,1.97,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),",",2.23,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),where,1.94,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
B-PER,B-PER (1.00),Emmanuel,1.17,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
I-PER,I-PER (1.00),Mac,1.27,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"


### Ignore indexes

To save computation time, we can indicate a list of token indexes that we want to ignore. The explainer will not compute explanations for these tokens, although attributions of these tokens will be calculated to explain the predictions over other tokens.

In [7]:
attributions_2 = ner_explainer(sample_text, ignored_indexes=[0, 1, 2, 11, 12, 13, 14])

When calling *visualize()*, we can see how the tokens corresponding to the ignored indexes do not appear.

In [8]:
html = ner_explainer.visualize()

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
B-LOC,B-LOC (1.00),Paris,1.03,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),last,1.46,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),weekend,1.97,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),",",2.23,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),where,1.94,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
B-PER,B-PER (1.00),Emmanuel,1.17,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
I-PER,I-PER (1.00),Mac,1.27,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
I-PER,I-PER (1.00),##ron,1.57,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"


### Ignore labels

In a similar way, we can also tell the explainer to ignore certain labels, e.g. we might not be interested in seeing the explanations of those tokens that are classified as *'O'*.

In [9]:
attributions_3 = ner_explainer(sample_text, ignored_labels=['O'])

In [10]:
attributions_3

{'Paris': {'label': 'B-LOC',
  'attribution_scores': [('[CLS]', 0.0),
   ('We', -0.026176158026416003),
   ('visited', 0.1289424214695528),
   ('Paris', 0.9820248557100411),
   ('last', -0.13107116600328042),
   ('weekend', 0.010904143358261256),
   (',', 0.011178508562829472),
   ('where', 0.023509729193194558),
   ('Emmanuel', 0.009830564279598433),
   ('Mac', 0.00034993661827931806),
   ('##ron', 0.014679480802688051),
   ('lives', 0.00523146911109651),
   ('.', 9.093164937093907e-05),
   ('[SEP]', 0.0)]},
 'Emmanuel': {'label': 'B-PER',
  'attribution_scores': [('[CLS]', 0.0),
   ('We', -0.02795400955500282),
   ('visited', -0.019536101884928644),
   ('Paris', 0.016961229547365956),
   ('last', 0.02993168457599209),
   ('weekend', -0.0005594575778221615),
   (',', -0.0028076008557282677),
   ('where', 0.4016979194311053),
   ('Emmanuel', 0.9076506398659525),
   ('Mac', -0.09408993595706144),
   ('##ron', -0.04634243847125355),
   ('lives', 0.02766440864687646),
   ('.', -0.02623680

In [11]:
html = ner_explainer.visualize()

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
B-LOC,B-LOC (1.00),Paris,1.03,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
B-PER,B-PER (1.00),Emmanuel,1.17,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
I-PER,I-PER (1.00),Mac,1.27,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
I-PER,I-PER (1.00),##ron,1.57,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"


The *ignored_labels* parameter can be combined with the *ignored_indexes* parameter.

In [14]:
attributions_4 = ner_explainer(sample_text, ignored_indexes=[11, 12, 13, 14], ignored_labels=['B-PER'])

In [15]:
html = ner_explainer.visualize()

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
O,O (1.00),[CLS],1.13,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),We,0.16,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),visited,1.15,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
B-LOC,B-LOC (1.00),Paris,1.03,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),last,1.46,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),weekend,1.97,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),",",2.23,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
O,O (1.00),where,1.94,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
I-PER,I-PER (1.00),Mac,1.27,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"
I-PER,I-PER (1.00),##ron,1.57,"[CLS] We visited Paris last weekend , where Emmanuel Mac ##ron lives . [SEP]"


As we can see, the tokens are ignored based on the predicted class and its index. The token `Emmanuel` is wrongly predicted as `B-PER`and is therefore ignored along with the tokens in positions `11`, `12`, `13` and `14`.